In [ ]:
# Protoype 1
# Control-click the file to see it.
"""/content/chessiewow.jpeg"""
"""/content/IMG_20241108_081319.jpg"""
"""/content/IMG_20241108_081525.jpg"""
"""/content/IMG_20241108_081707.jpg"""
"""/content/cell_phone.png"""
"""/content/floppy_disk.jpeg"""
print("Please be patient. It will take a long time, around 50-70 seconds. It might work better with bigger and higher-quality images.")
path_to_image = input("Image path:")
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import plot_model # Ignore possible error
import datetime

# Load a pre-trained object detection model
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

# Helper functions for image pre-processing and visualization

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3) # Ignore error
  return img

def run_detector(detector, path):
    img = load_img(path)

    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)

    result = {key:value.numpy() for key,value in result.items()}

    return img, result


def display_image_with_boxes(img, result, threshold=0.1):
    # Filter by threshold
    high_confidence_indices = np.where(result['detection_scores'] > threshold)[0]

    img_np = img.numpy().astype(np.uint8) # Convert image tensor to NumPy for OpenCV

    for i in high_confidence_indices:
        ymin, xmin, ymax, xmax = tuple(result['detection_boxes'][i])
        class_id = result['detection_class_entities'][i].decode("ascii")
        score = result['detection_scores'][i]


        # Calculate bounding box pixel coordinates
        xmin = int(xmin * img_np.shape[1])
        xmax = int(xmax * img_np.shape[1])
        ymin = int(ymin * img_np.shape[0])
        ymax = int(ymax * img_np.shape[0])

        cv2.rectangle(img_np, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        label = f"{class_id}: {score:.2f}"
        cv2.putText(img_np, label, (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(img_np)
    plt.show()



image_path = path_to_image
image, result = run_detector(detector, image_path)


display_image_with_boxes(image, result, threshold=0.25)


#-------------Tensorboard Logging-------------

def create_summary_writers(log_dir):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = log_dir + current_time + '/train'
    val_log_dir = log_dir + current_time + '/val'  # You'd use this if you were training

    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    # val_summary_writer = tf.summary.create_file_writer(val_log_dir) # uncomment if needed

    return train_summary_writer #, val_summary_writer


log_dir = "logs/"  # Define your log directory
train_summary_writer = create_summary_writers(log_dir)

# Example of how to log an image to TensorBoard
# This would typically happen in a training loop, but we demonstrate it here
img_tensor = tf.image.convert_image_dtype(image, dtype=tf.uint8)  # Make sure it's uint8 for TensorBoard.
with train_summary_writer.as_default():
    tf.summary.image("Input Image", img_tensor[tf.newaxis, ...], step=0)  # step=epoch or iteration number



print("TensorBoard logs written to:", log_dir)
print("To view TensorBoard, run: tensorboard --logdir=", log_dir)

